In [2]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.optimize import minimize

In [3]:
# load saved files
caps = pd.read_csv('stocks_market_caps.csv', index_col = 'Ticker')
stock_monthly = pd.read_csv('stock_monthly.csv', index_col = 0)
# 这两个退市

caps.drop('BRK.B', inplace = True)
caps.drop('BF.B', inplace = True)
caps.drop('LVS', inplace = True)
# 这三个新上市
caps.drop('VLTO', inplace = True)
caps.drop('KVUE', inplace = True)
caps.drop('GEHC', inplace = True)

# ticker index
sp500_tickers = caps.index

index_return = pd.read_csv('index_monthly.csv', index_col = 0)
time = index_return.index.tolist()
# use recent years
one_year = time[6:18]

In [12]:
def tracking_error(weights, target_returns, asset_returns):
    """
    Calculate tracking error

    Parameters:
    - weights:asset weights.
    - target_returns: index fund return
    - asset_returns: DataFrame containing asset returns.

    Returns:
    - tracking_error
    """
    portfolio_returns = np.dot(asset_returns, weights)
    tracking_error = np.sum((portfolio_returns - target_returns)**2)
    return tracking_error

In [13]:
def minimize_tracking_error(target_returns, asset_returns, max_stock_proportion):
    """
    Minimize the tracking error to a target return with constraints.

    Parameters:
    - target_return: Index return.
    - asset_returns: DataFrame containing asset returns.
    - max_stock_proportion: Maximum allowed proportion of stock holdings.

    Returns:
    - dict: A dictionary containing the optimal weights and minimum tracking error.
    """
    asset_returns = asset_returns.squeeze()  # Convert DataFrame to Series
    
    objective_function = lambda weights: tracking_error(weights, target_returns, asset_returns)
    
    # Define equality constraint (weights sum to 1)
    # single stock holding cannot exceed max_stock_proportion
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: max_stock_proportion - np.max(w)}] 
    

    
    # Define bounds for weights (between 0 and 1)
    bounds = [(0, 1) for _ in range(len(asset_returns))]

    weights = np.ones(len(asset_returns)) / len(asset_returns)

    # Run the optimization
    result = minimize(objective_function, weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Extract optimal weights and minimum tracking error from the result
    optimal_weights = result.x
    min_tracking_error = result.fun
    
    return {'weights': optimal_weights, 'min_tracking_error': min_tracking_error}


In [14]:
def index_fund(one_year, k_no, subset, proportion, adjusted): 
    portfolio_rts = []
    index_rts = []
    
    for i in range(len(one_year)):

        # data process
        date = one_year[i]
        df1 = pd.DataFrame()
        volas = pd.Series(dtype='object')
        returns  = pd.Series(dtype='object')
        val = stock_monthly.loc[date]
        for t in sp500_tickers:
            volas[t] = val[t+' Vola']
            returns[t] = val[t+ ' Return']
        df1['Vola'] = volas
        df1['Return'] = returns
        data = pd.concat([caps, df1], axis=1)
        index_rt = index_return['SP500 Return'][date] 
        
        # if not first month, start examning performance!        
        if i > 0:
            print("the portfolio weight is: \n", Weight_2)
            portfolio_rt = 0
            for stock in Weight_2.index:
                portfolio_rt +=  Weight_2[stock]*data['Return'][stock]
                
            index_rt = index_return['SP500 Return'][date] 
            print(date, "portfolio return is: ",portfolio_rt, " and index return is: ", index_rt)
            portfolio_rts.append(portfolio_rt)
            index_rts.append(index_rt)
          
        # last month, no need to calculate, because cannot compare, just report yearly tracking error
        if i == (len(one_year)-1):
            # Convert lists to pandas Series
            portfolio_series = pd.Series(portfolio_rts)
            index_series = pd.Series(index_rts)

            # Calculate the differences
            differences = portfolio_series - index_series

            # Calculate the tracking error as the standard deviation of the differences
            tracking_error = differences.std()

            print("The yearly tracking error is", tracking_error)
            break;

        # Finding the index of NaN values
    #     nan_indexes = df1[df1.isna().any(axis=1)].index

    #     print("Indexes with NaN values:", date, nan_indexes)
        scaler = StandardScaler()
        data_subset = data[subset]
        data_scaled_subset = scaler.fit_transform(data_subset)

        # Create and fit the KMeans model
        kmeans = KMeans(n_clusters=k_no, n_init=10)
        kmeans.fit(data_scaled_subset)

        # Get centroids
        centroids = kmeans.cluster_centers_
        
    # Initialize a dictionary to store the closest points
        closest_points = {}

        for i, centroid in enumerate(centroids):
            # Calculate distances of all points in df1_scaled to this centroid
            distances = np.linalg.norm(data_scaled_subset - centroid, axis=1)
        
            # Find the index of the minimum distance
            closest_point_idx = np.argmin(distances)
            
        # Map this index to the original DataFrame's index
            closest_points[f'Centroid_{i}'] = df1.index[closest_point_idx]
        
        result_centroid = []
        # Print the closest points to each centroid
        for centroid, point in closest_points.items():
            result_centroid.append(point)
        
        print(date, "the portfolio is: \n",result_centroid)
        #     print(f"{centroid}: Closest point is '{point}' from the original DataFrame.")
        df_filtered = data.loc[result_centroid]
        asset_returns = df_filtered[['Return']]
        target_return = index_rt
        result = minimize_tracking_error(target_return, asset_returns,proportion)
        #print(result['weights'])
        
        Weight_2 = pd.DataFrame({ 'Weight': result['weights']},index = result_centroid)
        Weight_2 = Weight_2['Weight'].sort_values(ascending=False)
        
      

In [16]:
index_fund(one_year, 50,['Vola', 'Return'], 0.05, True)

2022-12-31 the portfolio is: 
 ['NXPI', 'MOH', 'WY', 'TRGP', 'BX', 'LHX', 'AVGO', 'WBD', 'CNP', 'MRNA', 'CRL', 'CDAY', 'TSLA', 'VRSN', 'QCOM', 'EPAM', 'NCLH', 'ROL', 'DFS', 'HAL', 'ECL', 'CAG', 'FANG', 'LUV', 'SEE', 'SO', 'OKE', 'SYK', 'ILMN', 'MOS', 'NRG', 'DE', 'EL', 'KIM', 'PARA', 'APA', 'FOXA', 'NUE', 'MMM', 'FCX', 'AKAM', 'GOOG', 'BIO', 'TDG', 'AWK', 'WTW', 'CHTR', 'CTAS', 'SLB', 'PANW']
the portfolio weight is: 
 SO      0.022707
EL      0.022631
SYK     0.022550
HAL     0.022405
SLB     0.022374
VRSN    0.022101
AVGO    0.021991
MRNA    0.021945
CAG     0.021898
BIO     0.021787
AWK     0.021576
TDG     0.021521
WTW     0.021414
APA     0.021401
TRGP    0.021217
OKE     0.021080
MOH     0.021053
CTAS    0.020997
ECL     0.020933
DE      0.020930
CNP     0.020689
FCX     0.020486
CRL     0.020454
WY      0.020446
MMM     0.020424
SEE     0.020174
CDAY    0.020101
FOXA    0.020070
KIM     0.020025
ILMN    0.019878
FANG    0.019810
LHX     0.019652
ROL     0.019359
NXPI    0.019348